# Coordinate operations

In [1]:
import aeolus
aeolus.__version__

'0.4.10+8.gacd1df8.dirty'

For convenience, `aeolus` provides a way of keeping loaded and processed data within one object along with extra metadata.
The object is called `AtmoSim` (as in "atmospheric simulation"). 
The code below provides an example of basic usage of `AtmoSim`.

In [2]:
from pathlib import Path

import iris
from aeolus.core import AtmoSim

Note that you can use either a single filename or a list of filenames, each of which is either a `str` or (recommended) `pathlib.Path` object.

In [3]:
sample_file = Path.cwd() / "data" / "test_data" / "pp" / "sample_t1e_2d_mean.pp"

To load the data, use `iris` functions.

In [4]:
inp_data = iris.load(str(sample_file))

This cube list can be then used to instantiate `AtmoSim`. It is also possible to add a short name, a long description of the experiment; and to specify a planet configuration with relevant constants (see "Physical constants" example for more info).

In [5]:
my_run = AtmoSim(
    cubes=inp_data,
    name="t1e_example",
    description="This is some sample data from a UM simulation of tidally-locked Trappist-1e planet.",
    planet="trap1e",  # this reads constants from a JSON file
)

In [6]:
my_run

aeolus.core AtmoSim 't1e_example' [7 cubes]

In [14]:
repr(my_run.cv_rain)

"<iris 'Cube' of convective_rainfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)>"

In [125]:
class AS(AtmoSim):
    def __init__(self, *args, **kw):
        super(AtmoSim, self).__init__(*args, **kw)

        self.repr = ReprAtmoSimBase(self)

    def __repr__(self):  # noqa
        return self.repr.str_repr(short=True)

    def __str__(self):  # noqa
        return self.repr.str_repr(short=False)

    def _repr_html_(self):
        return self.repr.html_repr()


class ReprAtmoSimBase:
    _template = """
<style>
  table.aeolus {{
      white-space: pre;
      border: 1px solid;
      border-color: #f9f9ef;
      font-family: monaco, monospace;
  }}
  th.aeolus {{
      background: #084469;
      color: #fefefe;
      border-left: 1px solid;
      border-color: #0a507a;
      font-size: 1.05em;
      min-width: 50px;
      max-width: 125px;
  }}
  .aeolus-la {{
      text-align: left !important;
      white-space: pre;
  }}
  .aeolus-ra {{
      text-align: right !important;
      white-space: pre;
  }}
</style>
<table class="aeolus" id="{id}">
    {header}
    {content}
</table>
        """

    def __init__(self, atmosim):
        """
        Initialise ReprAtmoSimBase.

        Parameters
        ----------
        atmosim: aeolus.core.AtmoSimBase
            AtmoSimBase-like instance
        """
        self._cls_m = atmosim.__module__
        self._cls_n = atmosim.__class__.__name__
        self.as_id = id(atmosim)
        self.atmosim = atmosim
        self._cubes = self.atmosim._cubes
        self.name = f"{self._cls_m} {self._cls_n} '{self.atmosim.name}'"
        self._copy_attrs = [
            "description",
            "planet",
            "model",
            "model_type",
            "timestep",
            "domain",
            "vert_coord",
        ]
        self._max_len = max([len(i) for i in self._copy_attrs])
        for attr in self._copy_attrs:
            setattr(self, attr, getattr(atmosim, attr))

    def str_repr(self, short=False):
        """Represent AtmoSimBase as string."""
        summary = []
        summary.append(f"{self.name} [{len(self._cubes)} cubes]")
        # Short option - for __repr__
        if short:
            return " ".join(summary)
        # Long option - for __str__
        for attr in self._copy_attrs:
            if getattr(self, attr) is not None:
                summary.append(f"{attr:>{self._max_len}} | {getattr(self, attr)}")
        if len(self._cubes) > 0:
            summary.append("{!s}\n".format(self._cubes))
        return "\n".join(summary)

    def _make_header(self):
        top_cell = f'<th class="aeolus aeolus-la" colspan="2">{self.name}</th>'
        cells = ['<tr class="aeolus">', top_cell, "</tr>"]
        return "\n".join(cells)

    def _make_content(self):
        cells = []
        for attr in self._copy_attrs:
            if getattr(self, attr) is not None:
                value = getattr(self, attr)
                cells.append('<tr class="aeolus">')
                cells.append(f'<td class="aeolus aeolus-ra">{attr}</td>')
                cells.append(
                    f'<td class="aeolus aeolus-la">{value}</td>'
                )
                cells.append("</tr>")
        if len(self._cubes) > 0:
            # List cubes
            cells.append('<tr class="aeolus">')
            cells.append(f'<td class="octant aeolus-ra">Cubes</td>')
            cells.append(
                f'<td class="aeolus aeolus-la">{self._cubes._repr_html_()}</td>'
            )
            cells.append("</tr>")

        return "\n".join(cells)

    def html_repr(self):
        """HTML representation used in Jupyter Notebooks."""
        header = self._make_header()
        content = self._make_content()
        return self._template.format(id=self.as_id, header=header, content=content)

In [126]:
a = AS.from_parent_class(my_run)

In [129]:
a.model

<class 'aeolus.model.base.Model'>(80 fields)

In [68]:
print(a)

__main__ AS 't1e_example' [7 cubes]
0: convective_rainfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)
1: convective_snowfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)
2: high_type_cloud_area_fraction / (1) (latitude: 90; longitude: 144)
3: low_type_cloud_area_fraction / (1)  (latitude: 90; longitude: 144)
4: medium_type_cloud_area_fraction / (1) (latitude: 90; longitude: 144)
5: stratiform_rainfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)
6: stratiform_snowfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)

description | This is some sample data from a UM simulation of tidally-locked Trappist-1e planet.
     planet | trap1e
      model | <class 'aeolus.model.base.Model'>(80 fields)
     domain | Geographical region 't1e_example_domain' (west=-178.75, east=178.75, south=-89.0, north=89.0)


In [84]:
inp_data

[<iris 'Cube' of convective_rainfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)>,
<iris 'Cube' of convective_snowfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)>,
<iris 'Cube' of high_type_cloud_area_fraction / (1) (latitude: 90; longitude: 144)>,
<iris 'Cube' of low_type_cloud_area_fraction / (1) (latitude: 90; longitude: 144)>,
<iris 'Cube' of medium_type_cloud_area_fraction / (1) (latitude: 90; longitude: 144)>,
<iris 'Cube' of stratiform_rainfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)>,
<iris 'Cube' of stratiform_snowfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)>]

In [75]:
from iris.experimental.representation import CubeListRepresentation, CubeRepresentation

In [76]:
CubeRepresentation??

Init signature: CubeRepresentation(cube)
Source:        
class CubeRepresentation:
    """
    Produce representations of a :class:`~iris.cube.Cube`.

    This includes:

    * ``_html_repr_``: a representation of the cube as an html object,
      available in Jupyter notebooks. Specifically, this is presented as an
      html table.

    """

    _template = """
<style>
  a.iris {{
      text-decoration: none !important;
  }}
  table.iris {{
      white-space: pre;
      border: 1px solid;
      border-color: #9c9c9c;
      font-family: monaco, monospace;
  }}
  th.iris {{
      background: #303f3f;
      color: #e0e0e0;
      border-left: 1px solid;
      border-color: #9c9c9c;
      font-size: 1.05em;
      min-width: 50px;
      max-width: 125px;
  }}
  tr.iris :first-child {{
      border-right: 1px solid #9c9c9c !important;
  }}
  td.iris-title {{
      background: #d5dcdf;
      border-top: 1px solid #9c9c9c;
      font-weight: bold;
  }}
  .iris-word-cell {{
      text-align: l

In [6]:
type(my_run)

aeolus.core.AtmoSim

The loaded data are stored as a `CubeList` as the `_cubes` attribute.

In [7]:
print(my_run._cubes)

0: convective_rainfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)
1: convective_snowfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)
2: high_type_cloud_area_fraction / (1) (latitude: 90; longitude: 144)
3: low_type_cloud_area_fraction / (1)  (latitude: 90; longitude: 144)
4: medium_type_cloud_area_fraction / (1) (latitude: 90; longitude: 144)
5: stratiform_rainfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)
6: stratiform_snowfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)


Constants that have been used in the model:

In [8]:
my_run.const

Trap1eConstants(earth_day [s], stefan_boltzmann [W m-2 K-4], water_heat_vaporization [m2 s-2], water_molecular_weight [kg mol-1], molar_gas_constant [J K-1 mol-1], boltzmann [m^2 kg s^-2 K^-1], avogadro [mol-1], gravity [m s-2], radius [m], day [s], solar_constant [W m-2], reference_surface_pressure [Pa], semi_major_axis [au], eccentricity [1], obliquity [degree], dry_air_spec_heat_press [m2 s-2 K-1], dry_air_molecular_weight [kg mol-1], condensible_density [kg m-3], condensible_heat_vaporization [m2 s-2])

Individual cubes can be accessed either by via the `_cubes` attribute and `iris` methods, or, more conveniently, as keys or attributes of `AtmoSim`:

In [9]:
print(my_run.cv_rain)

convective_rainfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)
     Dimension coordinates:
          latitude                               x              -
          longitude                              -              x
     Scalar coordinates:
          forecast_period: 36360.0 hours, bound=(36000.0, 36720.0) hours
          forecast_reference_time: 2004-05-13 09:00:00
          time: 2008-07-06 09:00:00, bound=(2008-06-21 09:00:00, 2008-07-21 09:00:00)
     Attributes:
          STASH: m01s05i205
          planet_conf: Trap1eConstants(earth_day [s], stefan_boltzmann [W m-2 K-4], water_heat_vaporization...
          source: Data from Met Office Unified Model
          um_version: 11.3
     Cell methods:
          mean: time (1 hour)


In [10]:
print(my_run["cv_rain"])

convective_rainfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)
     Dimension coordinates:
          latitude                               x              -
          longitude                              -              x
     Scalar coordinates:
          forecast_period: 36360.0 hours, bound=(36000.0, 36720.0) hours
          forecast_reference_time: 2004-05-13 09:00:00
          time: 2008-07-06 09:00:00, bound=(2008-06-21 09:00:00, 2008-07-21 09:00:00)
     Attributes:
          STASH: m01s05i205
          planet_conf: Trap1eConstants(earth_day [s], stefan_boltzmann [W m-2 K-4], water_heat_vaporization...
          source: Data from Met Office Unified Model
          um_version: 11.3
     Cell methods:
          mean: time (1 hour)


which is equivalent to typing

In [11]:
print(my_run._cubes.extract_cube("convective_rainfall_flux"))

convective_rainfall_flux / (kg m-2 s-1) (latitude: 90; longitude: 144)
     Dimension coordinates:
          latitude                               x              -
          longitude                              -              x
     Scalar coordinates:
          forecast_period: 36360.0 hours, bound=(36000.0, 36720.0) hours
          forecast_reference_time: 2004-05-13 09:00:00
          time: 2008-07-06 09:00:00, bound=(2008-06-21 09:00:00, 2008-07-21 09:00:00)
     Attributes:
          STASH: m01s05i205
          planet_conf: Trap1eConstants(earth_day [s], stefan_boltzmann [W m-2 K-4], water_heat_vaporization...
          source: Data from Met Office Unified Model
          um_version: 11.3
     Cell methods:
          mean: time (1 hour)


Short-cuts are defined using the `Model` container (see the "Model field names" example).

In [12]:
my_run.model.cv_rain

'convective_rainfall_flux'